# 1. Import some packages

* Numpy
* TensorFlow / Keras
* Sklearn
* Os
* Pandas
* Zipfile
* OpenCV
* Matplotlib
* Time
* Mat73
* Tqdm
* Gc
* Classifiers
* Among others

In [ ]:
#Install packages before import

!pip install mat73
!pip install git+https://github.com/qubvel/classification_models.git
!pip install -U segmentation-models
!pip install -U albumentations
!pip install openpyxl

In [ ]:
# Importing packages

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau,ModelCheckpoint,EarlyStopping 
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import os
import pandas as pd
import zipfile
import cv2
import matplotlib.pyplot as plt
import datetime
import mat73
from tqdm import tqdm
import gc
from classification_models.tfkeras import Classifiers
import random
import scipy.stats as stats
from scipy.stats import shapiro
import seaborn as sns
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import MultiComparison
import albumentations as A
%env SM_FRAMEWORK=tf.keras
import segmentation_models as sm
sm.set_framework('tf.keras')

# 2. Loading data and preprocessing a little bit
Brain tumor data

In [ ]:
# Access the path, load and preprocessing the informations

images_dir_1 = os.path.join('../input/brain-tumor-dataset-original/brainTumorDataPublic_1-766/')
images_dir_2 = os.path.join('../input/brain-tumor-dataset-original/brainTumorDataPublic_1533-2298/')
images_dir_3 = os.path.join('../input/brain-tumor-dataset-original/brainTumorDataPublic_2299-3064/')
images_dir_4 = os.path.join('../input/brain-tumor-dataset-original/brainTumorDataPublic_767-1532/')

images_list_1 = os.listdir(images_dir_1)
images_list_2 = os.listdir(images_dir_2)
images_list_3 = os.listdir(images_dir_3)
images_list_4 = os.listdir(images_dir_4)

images_dir = ['../input/brain-tumor-dataset-original/brainTumorDataPublic_1-766/', 
              '../input/brain-tumor-dataset-original/brainTumorDataPublic_1533-2298/', 
              '../input/brain-tumor-dataset-original/brainTumorDataPublic_2299-3064/', 
              '../input/brain-tumor-dataset-original/brainTumorDataPublic_767-1532/']
images_list = [images_list_1, images_list_2, images_list_3, images_list_4]

images = []
masks = []
labels = []

for i,j in zip(images_dir, images_list):
    for k in tqdm(j):
        path_image = str(i) + str(k)
        data_dict = mat73.loadmat(path_image)
        images.append((data_dict['cjdata']['image'])) #This is a problematic part
        masks.append((data_dict['cjdata']['tumorMask']*1).astype('uint8'))
        labels = np.append(labels, data_dict['cjdata']['label'])
print('There are {} images.'.format(len(images)))

In [ ]:
# Preprocessing the label

le = LabelEncoder()
labels_labelencoder = pd.DataFrame(np.array(labels).T).apply(le.fit_transform)
enc = OneHotEncoder()
enc.fit(labels_labelencoder)
labels_labelencoder_onehotencoder = enc.transform(labels_labelencoder).toarray()
labels_labelencoder_onehotencoder

In [ ]:
#What is data_dict?

data_dict

In [ ]:
# Deleting useless data

del data_dict
del labels
del labels_labelencoder
gc.collect()

# 4. Spliting/preprocessing and saving the data into train/validation and test sets

**Classification**

In [ ]:
#Spliting

X_train_validation, X_test, y_train_validation, y_test = train_test_split(images, labels_labelencoder_onehotencoder, test_size=0.2, random_state=0)

# Deleting useless data

del images
del labels_labelencoder_onehotencoder
gc.collect()

#Saving

np.save('X_train_validation.npy', X_train_validation)
np.save('X_test.npy', X_test)
np.save('y_train_validation.npy',y_train_validation)
np.save('y_test.npy', y_test)

**Segmentation**

In [ ]:
#Spliting

X_train_validation, X_test, y_train_validation, y_test = train_test_split(images, masks, test_size=0.2, random_state=0)

# Deleting useless data

del images
del masks
gc.collect()

#Saving

np.save('X_train_validation.npy', X_train_validation)
np.save('X_test.npy', X_test)
np.save('y_train_validation.npy',y_train_validation)
np.save('y_test.npy', y_test)

In [ ]:
#Show images

f, axarr = plt.subplots(2,2)
axarr[0,0].imshow(X_train_validation[6], cmap='gray')
axarr[0,1].imshow(X_train_validation[7], cmap='gray')
axarr[1,0].imshow(y_train_validation[6], cmap='gray')
axarr[1,1].imshow(y_train_validation[7], cmap='gray')

# 5. Initializing brain tumor classification

In [ ]:
# Loading the saved data for classification - PNG method

os.makedirs("./X_train_validation_png")
os.makedirs("./X_test_png")

X_train_validation_jpg = np.load('../input/brain-tumor-dataset-modified/X_train_validation.npy', allow_pickle = True)
for i in tqdm(range(0,2451)):    
    X1 = cv2.convertScaleAbs(X_train_validation_jpg[i], alpha = 0.03).tolist()
    plt.imsave('./X_train_validation_png/image0' + str(i) + '.png', X1, cmap='gray')
    

X_test_jpg = np.load('../input/brain-tumor-dataset-modified/X_test.npy', allow_pickle = True)
for i in tqdm(range(0,613)):    
    X2 = cv2.convertScaleAbs(X_test_jpg[i], alpha = 0.03).tolist()
    plt.imsave('./X_test_png/image0' + str(i) + '.png', X2, cmap='gray')
    
y_train_validation = np.load('../input/brain-tumor-dataset-modified/y_train_validation.npy', allow_pickle = True)
y_test = np.load('../input/brain-tumor-dataset-modified/y_test.npy', allow_pickle = True)

In [ ]:
# Save zipfile of X_train_validation

import os
import zipfile
 
fantasy_zip = zipfile.ZipFile('./X_train_validation_png.zip', 'w')
 
for folder, subfolders, files in os.walk('./X_train_validation_png'):
 
    for file in files:
        if file.endswith('.png'):
            fantasy_zip.write(os.path.join(folder, file), os.path.relpath(os.path.join(folder,file), './PNEUMOTHORAX'), compress_type = zipfile.ZIP_DEFLATED)
 
fantasy_zip.close()

In [ ]:
# Save zipfile of X_test

import os
import zipfile
 
fantasy_zip = zipfile.ZipFile('./X_test_png.zip', 'w')
 
for folder, subfolders, files in os.walk('./X_test_png'):
 
    for file in files:
        if file.endswith('.png'):
            fantasy_zip.write(os.path.join(folder, file), os.path.relpath(os.path.join(folder,file), './PNEUMOTHORAX'), compress_type = zipfile.ZIP_DEFLATED)
 
fantasy_zip.close()

Good point to restart

In [ ]:
# Read image and resize using INTER_AREA

x_train_validation_data = list()
x_test_data = list()

for number in tqdm(range(0,2451)):
    im = cv2.imread('../input/images-grayscale/X_train_validation_png/image0' + str(number) + '.png',cv2.IMREAD_GRAYSCALE)
    im = cv2.resize(im, (256,256), interpolation = cv2.INTER_AREA)
    x_train_validation_data.append(im)
for number in tqdm(range(0,613)):
    im = cv2.imread('../input/images-grayscale/X_test_png/image0' + str(number) + '.png',cv2.IMREAD_GRAYSCALE)
    im = cv2.resize(im, (256,256), interpolation = cv2.INTER_AREA)
    x_test_data.append(im)

In [ ]:
#Loading the y data
y_train_validation = np.load('../input/brain-tumor-dataset-modified/y_train_validation.npy', allow_pickle = True)
y_test = np.load('../input/brain-tumor-dataset-modified/y_test.npy', allow_pickle = True)

In [ ]:
#Show images

f, axarr = plt.subplots(2,2)
axarr[0,0].imshow(x_train_validation_data[2441], cmap='gray')
axarr[0,1].imshow(x_train_validation_data[2449], cmap='gray')
axarr[1,0].imshow(x_train_validation_data[6], cmap='gray')
axarr[1,1].imshow(x_train_validation_data[7], cmap='gray')

In [ ]:
#Reshape the images

x_train_validation_data = (np.array(x_train_validation_data).reshape((len(x_train_validation_data), 256, 256, 1))).astype('float16') / 255
x_test_data = (np.array(x_test_data).reshape((len(x_test_data), 256, 256, 1))).astype('float16') / 255

In [ ]:
# Activating the TPU

tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
# Classifiers avaliable in the package

Classifiers.models_names()

In [ ]:
from tensorflow.keras import backend as K

def custom_f1(y_true, y_pred):    
    def recall_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        
        recall = TP / (Positives+K.epsilon())    
        return recall 
    
    
    def precision_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    
        precision = TP / (Pred_Positives+K.epsilon())
        return precision 
    
    precision, recall = precision_m(y_true, y_pred), recall_m(y_true, y_pred)
    
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
# Constructing the NN and executing the tran and validation
start_time = datetime.datetime.now()

kfold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
resultados = []
j = 0

fpr = [0] * 10
for i in range(10):
    fpr[i] = [0] * 3
    
tpr = [0] * 10
for i in range(10):
    tpr[i] = [0] * 3

for train_index, test_index in kfold.split(x_train_validation_data,np.zeros(shape = (x_train_validation_data.shape[0],1))):
    
    SIZE = len(x_train_validation_data[0])

    with tpu_strategy.scope():

        seresnext50, _ = Classifiers.get('mobilenet')

        input_tensor = keras.layers.Input(shape=(SIZE, SIZE,1))
        x = keras.layers.Conv2D(3,(1,1),padding='same')(input_tensor)
        base_model = seresnext50(input_shape = (SIZE, SIZE,3), include_top = False, weights = 'imagenet')(x)
        y = keras.layers.GlobalAveragePooling2D()(base_model)
        y2 = tf.keras.layers.BatchNormalization()(y)
        y3 = tf.keras.layers.Flatten()(y2)
        y4 = tf.keras.layers.Dense(1026, activation="relu")(y3)
        y5 = tf.keras.layers.Dropout(0.5)(y4)
        output = keras.layers.Dense(3, activation='softmax')(y5)
        model = keras.models.Model(inputs=[input_tensor], outputs=[output])
        model.summary()

        model.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=[tf.keras.metrics.AUC(multi_label=True), custom_f1,'accuracy',
                                                                                 tf.keras.metrics.Precision(), keras.metrics.Recall(),tf.keras.metrics.TruePositives(), tf.keras.metrics.TrueNegatives(), tf.keras.metrics.FalsePositives(), tf.keras.metrics.FalseNegatives()])

        if j == 0:
            k = 'val_custom_f1'
        else:
            k = 'val_custom_f1'#_'+str(j)

        name= 'MRI_mobilenet.h5'
        rlr = ReduceLROnPlateau(monitor = k, factor = 0.1, patience = 3, verbose = 1, min_delta = 1e-4, min_lr = 1e-6, mode = 'max', cooldown=1)
        ckp = ModelCheckpoint(name,monitor = k,verbose = 1, save_best_only = True, mode = 'max')    
        es = EarlyStopping(monitor = k, min_delta = 1e-4, patience =10, mode = 'max', restore_best_weights = True, verbose = 1)
        history = model.fit(x_train_validation_data[train_index], y_train_validation[train_index],
                            batch_size = 64, epochs = 100, 
                            validation_data = (x_train_validation_data[test_index], y_train_validation[test_index]),
                            callbacks=[rlr,es,ckp])
        
        dependencies = {'custom_f1': custom_f1}
        model_= tf.keras.models.load_model('./MRI_mobilenet.h5', custom_objects=dependencies)
        model_.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=[tf.keras.metrics.AUC(multi_label=True), custom_f1,'accuracy',
                                                                          tf.keras.metrics.Precision(), keras.metrics.Recall(),tf.keras.metrics.TruePositives(), tf.keras.metrics.TrueNegatives(), tf.keras.metrics.FalsePositives(), tf.keras.metrics.FalseNegatives()])
        precision = model_.evaluate(x_train_validation_data[test_index],
                                    y_train_validation[test_index],
                                    batch_size = 64)
        
        
        resultados.append(precision)
        j += 1
        
end_time = datetime.datetime.now()
print('Duration: {}'.format(end_time - start_time))

In [ ]:
resultados

Here you can start by now

In [ ]:
alpha = 0.05
mean_class = pd.read_csv('../input/mean-class-ok/Mdias_Class.csv')
mean_class

In [ ]:
#AUC

#Variance Homogeneity       igualdade das var > 0.05
print(stats.levene(mean_class['AUC_Seresnext50'],mean_class['AUC_Vgg16'],mean_class['AUC_Mobilenet'], center = 'median'))

#Normality                  normalidade da distr. >0.05
print(shapiro(mean_class['AUC_Seresnext50']),shapiro(mean_class['AUC_Vgg16']),shapiro(mean_class['AUC_Mobilenet']))
sns.displot(mean_class['AUC_Seresnext50'], kind = 'kde');
sns.displot(mean_class['AUC_Vgg16'], kind = 'kde');
sns.displot(mean_class['AUC_Mobilenet'], kind = 'kde');

# <4 ANOVA anyway
# >4 Kruskal-Wallis Test
print(np.var(mean_class['AUC_Seresnext50']), np.var(mean_class['AUC_Vgg16']), np.var(mean_class['AUC_Mobilenet']))
print(3.1033555129181136e-06/4.332674250218629e-07)

#ANOVA
#_,p = f_oneway(mean_class['AUC_Seresnext50'],mean_class['AUC_Vgg16'],mean_class['AUC_Mobilenet'])
#Kruskal-Wallis    igual > 0.05
_,p = stats.kruskal(mean_class['AUC_Seresnext50'],mean_class['AUC_Vgg16'],mean_class['AUC_Mobilenet'])
print(p)

In [ ]:
#F1

#Variance Homogeneity       igualdade das var > 0.05
print(stats.levene(mean_class['F1_Seresnext50'],mean_class['F1_Vgg16'],mean_class['F1_Mobilenet'], center = 'median'))

#Normality                  normalidade da distr. >0.05
print(shapiro(mean_class['F1_Seresnext50']),shapiro(mean_class['F1_Vgg16']),shapiro(mean_class['F1_Mobilenet']))
sns.displot(mean_class['F1_Seresnext50'], kind = 'kde');
sns.displot(mean_class['F1_Vgg16'], kind = 'kde');
sns.displot(mean_class['F1_Mobilenet'], kind = 'kde');

# <4 ANOVA anyway
# >4 Kruskal-Wallis Test
print(np.var(mean_class['F1_Seresnext50']), np.var(mean_class['F1_Vgg16']), np.var(mean_class['F1_Mobilenet']))
print(4.2278554688608024e-05/3.966609370570272e-06)

#ANOVA
#_,p = f_oneway(mean_class['F1_Seresnext50'],mean_class['F1_Vgg16'],mean_class['F1_Mobilenet'])
#Kruskal-Wallis    igual > 0.05
_,p = stats.kruskal(mean_class['F1_Seresnext50'],mean_class['F1_Vgg16'],mean_class['F1_Mobilenet'])
print(p)

In [ ]:
#Acc

#Variance Homogeneity       igualdade das var > 0.05
print(stats.levene(mean_class['Acc_Seresnext50'],mean_class['Acc_Vgg16'],mean_class['Acc_Mobilenet'], center = 'median'))

#Normality                  normalidade da distr. >0.05
print(shapiro(mean_class['Acc_Seresnext50']),shapiro(mean_class['Acc_Vgg16']),shapiro(mean_class['Acc_Mobilenet']))
sns.displot(mean_class['Acc_Seresnext50'], kind = 'kde');
sns.displot(mean_class['Acc_Vgg16'], kind = 'kde');
sns.displot(mean_class['Acc_Mobilenet'], kind = 'kde');

# <4 ANOVA anyway
# >4 Kruskal-Wallis Test
print(np.var(mean_class['Acc_Seresnext50']), np.var(mean_class['Acc_Vgg16']), np.var(mean_class['Acc_Mobilenet']))
print(4.606542803178421e-05/3.7360412156123934e-06)

#ANOVA
#_,p = f_oneway(mean_class['Acc_Seresnext50'],mean_class['Acc_Vgg16'],mean_class['Acc_Mobilenet'])
#Kruskal-Wallis    igual > 0.05
_,p = stats.kruskal(mean_class['Acc_Seresnext50'],mean_class['Acc_Vgg16'],mean_class['Acc_Mobilenet'])
print(p)
#Há deferença
p1 = stats.kruskal(mean_class['Acc_Seresnext50'],mean_class['Acc_Vgg16'])
print(p1)
p2 = stats.kruskal(mean_class['Acc_Seresnext50'],mean_class['Acc_Mobilenet'])
print(p2)
p3 = stats.kruskal(mean_class['Acc_Vgg16'],mean_class['Acc_Mobilenet'])
print(p3)

In [ ]:
#Prec

#Variance Homogeneity       igualdade das var > 0.05
print(stats.levene(mean_class['Prec_Seresnext50'],mean_class['Prec_Vgg16'],mean_class['Prec_Mobilenet'], center = 'median'))

#Normality                  normalidade da distr. >0.05
print(shapiro(mean_class['Prec_Seresnext50']),shapiro(mean_class['Prec_Vgg16']),shapiro(mean_class['Prec_Mobilenet']))
sns.displot(mean_class['Prec_Seresnext50'], kind = 'kde');
sns.displot(mean_class['Prec_Vgg16'], kind = 'kde');
sns.displot(mean_class['Prec_Mobilenet'], kind = 'kde');

# <4 ANOVA anyway
# >4 Kruskal-Wallis Test
print(np.var(mean_class['Prec_Seresnext50']), np.var(mean_class['Prec_Vgg16']), np.var(mean_class['Prec_Mobilenet']))
print(2.5833736912516415e-05/4.0185712494159865e-06)

#ANOVA igual > 0.05
_,p = f_oneway(mean_class['Prec_Seresnext50'],mean_class['Prec_Vgg16'],mean_class['Prec_Mobilenet'])

#Kruskal-Wallis    igual > 0.05
#_,p = stats.kruskal(mean_class['Prec_Seresnext50'],mean_class['Prec_Vgg16'],mean_class['Prec_Mobilenet'])
print(p)

In [ ]:
#Rec

#Variance Homogeneity       igualdade das var > 0.05
print(stats.levene(mean_class['Rec_Seresnext50'],mean_class['Rec_Vgg16'],mean_class['Rec_Mobilenet'], center = 'median'))

#Normality                  normalidade da distr. >0.05
print(shapiro(mean_class['Rec_Seresnext50']),shapiro(mean_class['Rec_Vgg16']),shapiro(mean_class['Rec_Mobilenet']))
sns.displot(mean_class['Rec_Seresnext50'], kind = 'kde');
sns.displot(mean_class['Rec_Vgg16'], kind = 'kde');
sns.displot(mean_class['Rec_Mobilenet'], kind = 'kde');

# <4 ANOVA anyway
# >4 Kruskal-Wallis Test
print(np.var(mean_class['Rec_Seresnext50']), np.var(mean_class['Rec_Vgg16']), np.var(mean_class['Rec_Mobilenet']))
print(6.229484832056975e-05/3.99211178658067e-06)

#ANOVA
#_,p = f_oneway(mean_class['Rec_Seresnext50'],mean_class['Rec_Vgg16'],mean_class['Rec_Mobilenet'])
#Kruskal-Wallis    igual > 0.05
_,p = stats.kruskal(mean_class['Rec_Seresnext50'],mean_class['Rec_Vgg16'],mean_class['Rec_Mobilenet'])
print(p)
#Há deferença
p1 = stats.kruskal(mean_class['Rec_Seresnext50'],mean_class['Rec_Vgg16'])
print(p1)
p2 = stats.kruskal(mean_class['Rec_Seresnext50'],mean_class['Rec_Mobilenet'])
print(p2)
p3 = stats.kruskal(mean_class['Rec_Vgg16'],mean_class['Rec_Mobilenet'])
print(p3)

In [ ]:
#Variance Homogeneity

#AUC
stats.levene(mean_class['AUC_Seresnext50'],mean_class['AUC_Vgg16'],mean_class['AUC_Mobilenet'], center = 'median')

In [ ]:
#F1
stats.levene(mean_class['F1_Seresnext50'],mean_class['F1_Vgg16'],mean_class['F1_Mobilenet'], center = 'median')

In [ ]:
#Acc
stats.levene(mean_class['Acc_Seresnext50'],mean_class['Acc_Vgg16'],mean_class['Acc_Mobilenet'], center = 'median')

In [ ]:
#Precision
stats.levene(mean_class['Prec_Seresnext50'],mean_class['Prec_Vgg16'],mean_class['Prec_Mobilenet'], center = 'median')

In [ ]:
#Recall
stats.levene(mean_class['Rec_Seresnext50'],mean_class['Rec_Vgg16'],mean_class['Rec_Mobilenet'])

In [ ]:
#Normality

#AUC
shapiro(mean_class['AUC_Seresnext50']),shapiro(mean_class['AUC_Vgg16']),shapiro(mean_class['AUC_Mobilenet'])

In [ ]:
sns.displot(mean_class['AUC_Vgg16'], kind = 'kde');

In [ ]:
#F1
shapiro(mean_class['F1_Seresnext50']),shapiro(mean_class['F1_Vgg16']),shapiro(mean_class['F1_Mobilenet'])

In [ ]:
#Acc
shapiro(mean_class['Acc_Seresnext50']),shapiro(mean_class['Acc_Vgg16']),shapiro(mean_class['Acc_Mobilenet'])

In [ ]:
sns.displot(mean_class['Acc_Seresnext50'], kind = 'kde');

In [ ]:
#Precision
shapiro(mean_class['Prec_Seresnext50']),shapiro(mean_class['Prec_Vgg16']),shapiro(mean_class['Prec_Mobilenet'])

In [ ]:
#Recall
shapiro(mean_class['Rec_Seresnext50']),shapiro(mean_class['Rec_Vgg16']),shapiro(mean_class['Rec_Mobilenet'])

In [ ]:
_,p = f_oneway(mean_class['AUC_Seresnext50'],mean_class['AUC_Vgg16'],mean_class['AUC_Mobilenet'])
p

In [ ]:
_,p = f_oneway(mean_class['F1_Seresnext50'],mean_class['F1_Vgg16'],mean_class['F1_Mobilenet'])
p

In [ ]:
_,p = f_oneway(mean_class['Acc_Seresnext50'],mean_class['Acc_Vgg16'],mean_class['Acc_Mobilenet'])
p

In [ ]:
_,p = f_oneway(mean_class['Prec_Seresnext50'],mean_class['Prec_Vgg16'],mean_class['Prec_Mobilenet'])
p

In [ ]:
_,p = f_oneway(mean_class['Rec_Seresnext50'],mean_class['Rec_Vgg16'],mean_class['Rec_Mobilenet'])
p

In [ ]:
resultado_algorithm_AUC = {'AUC': np.concatenate([mean_class['AUC_Seresnext50'], mean_class['AUC_Vgg16'], mean_class['AUC_Mobilenet']]),
                         'algorithm': ['AUC_Seresnext50','AUC_Seresnext50','AUC_Seresnext50','AUC_Seresnext50','AUC_Seresnext50','AUC_Seresnext50','AUC_Seresnext50','AUC_Seresnext50','AUC_Seresnext50','AUC_Seresnext50',
                                      'AUC_Vgg16','AUC_Vgg16','AUC_Vgg16','AUC_Vgg16','AUC_Vgg16','AUC_Vgg16','AUC_Vgg16','AUC_Vgg16','AUC_Vgg16','AUC_Vgg16',
                                      'AUC_Mobilenet','AUC_Mobilenet','AUC_Mobilenet','AUC_Mobilenet','AUC_Mobilenet','AUC_Mobilenet','AUC_Mobilenet','AUC_Mobilenet','AUC_Mobilenet','AUC_Mobilenet']}
results = pd.DataFrame(resultado_algorithm_AUC)
compara_algoritmos = MultiComparison(results['AUC'],results['algorithm'])
teste_estatistico = compara_algoritmos.tukeyhsd()
print(teste_estatistico)

In [ ]:
resultado_algorithm_F1 = {'F1': np.concatenate([mean_class['F1_Seresnext50'], mean_class['F1_Vgg16'], mean_class['F1_Mobilenet']]),
                         'algorithm': ['F1_Seresnext50','F1_Seresnext50','F1_Seresnext50','F1_Seresnext50','F1_Seresnext50','F1_Seresnext50','F1_Seresnext50','F1_Seresnext50','F1_Seresnext50','F1_Seresnext50',
                                      'F1_Vgg16','F1_Vgg16','F1_Vgg16','F1_Vgg16','F1_Vgg16','F1_Vgg16','F1_Vgg16','F1_Vgg16','F1_Vgg16','F1_Vgg16',
                                      'F1_Mobilenet','F1_Mobilenet','F1_Mobilenet','F1_Mobilenet','F1_Mobilenet','F1_Mobilenet','F1_Mobilenet','F1_Mobilenet','F1_Mobilenet','F1_Mobilenet']}
results = pd.DataFrame(resultado_algorithm_F1)
compara_algoritmos = MultiComparison(results['F1'],results['algorithm'])
teste_estatistico = compara_algoritmos.tukeyhsd()
print(teste_estatistico)

In [ ]:
resultado_algorithm_Acc = {'Acc': np.concatenate([mean_class['Acc_Seresnext50'], mean_class['Acc_Vgg16'], mean_class['Acc_Mobilenet']]),
                         'algorithm': ['Acc_Seresnext50','Acc_Seresnext50','Acc_Seresnext50','Acc_Seresnext50','Acc_Seresnext50','Acc_Seresnext50','Acc_Seresnext50','Acc_Seresnext50','Acc_Seresnext50','Acc_Seresnext50',
                                      'Acc_Vgg16','Acc_Vgg16','Acc_Vgg16','Acc_Vgg16','Acc_Vgg16','Acc_Vgg16','Acc_Vgg16','Acc_Vgg16','Acc_Vgg16','Acc_Vgg16',
                                      'Acc_Mobilenet','Acc_Mobilenet','Acc_Mobilenet','Acc_Mobilenet','Acc_Mobilenet','Acc_Mobilenet','Acc_Mobilenet','Acc_Mobilenet','Acc_Mobilenet','Acc_Mobilenet']}
results = pd.DataFrame(resultado_algorithm_Acc)
compara_algoritmos = MultiComparison(results['Acc'],results['algorithm'])
teste_estatistico = compara_algoritmos.tukeyhsd()
print(teste_estatistico)

In [ ]:
resultado_algorithm_Prec = {'Prec': np.concatenate([mean_class['Prec_Seresnext50'], mean_class['Prec_Vgg16'], mean_class['Prec_Mobilenet']]),
                         'algorithm': ['Prec_Seresnext50','Prec_Seresnext50','Prec_Seresnext50','Prec_Seresnext50','Prec_Seresnext50','Prec_Seresnext50','Prec_Seresnext50','Prec_Seresnext50','Prec_Seresnext50','Prec_Seresnext50',
                                      'Prec_Vgg16','Prec_Vgg16','Prec_Vgg16','Prec_Vgg16','Prec_Vgg16','Prec_Vgg16','Prec_Vgg16','Prec_Vgg16','Prec_Vgg16','Prec_Vgg16',
                                      'Prec_Mobilenet','Prec_Mobilenet','Prec_Mobilenet','Prec_Mobilenet','Prec_Mobilenet','Prec_Mobilenet','Prec_Mobilenet','Prec_Mobilenet','Prec_Mobilenet','Prec_Mobilenet']}
results = pd.DataFrame(resultado_algorithm_Prec)
compara_algoritmos = MultiComparison(results['Prec'],results['algorithm'])
teste_estatistico = compara_algoritmos.tukeyhsd()
print(teste_estatistico)

In [ ]:
resultado_algorithm_Rec = {'Rec': np.concatenate([mean_class['Rec_Seresnext50'], mean_class['Rec_Vgg16'], mean_class['Rec_Mobilenet']]),
                         'algorithm': ['Rec_Seresnext50','Rec_Seresnext50','Rec_Seresnext50','Rec_Seresnext50','Rec_Seresnext50','Rec_Seresnext50','Rec_Seresnext50','Rec_Seresnext50','Rec_Seresnext50','Rec_Seresnext50',
                                      'Rec_Vgg16','Rec_Vgg16','Rec_Vgg16','Rec_Vgg16','Rec_Vgg16','Rec_Vgg16','Rec_Vgg16','Rec_Vgg16','Rec_Vgg16','Rec_Vgg16',
                                      'Rec_Mobilenet','Rec_Mobilenet','Rec_Mobilenet','Rec_Mobilenet','Rec_Mobilenet','Rec_Mobilenet','Rec_Mobilenet','Rec_Mobilenet','Rec_Mobilenet','Rec_Mobilenet']}
results = pd.DataFrame(resultado_algorithm_Rec)
compara_algoritmos = MultiComparison(results['Rec'],results['algorithm'])
teste_estatistico = compara_algoritmos.tukeyhsd()
print(teste_estatistico)

In [ ]:
# Evaluating the model - test
dependencies = {'custom_f1': custom_f1}
model_= tf.keras.models.load_model('./MRI_mobilenet.h5', custom_objects=dependencies)
model_.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=[tf.keras.metrics.AUC(multi_label=True), custom_f1,'accuracy',
                                                                          tf.keras.metrics.Precision(), keras.metrics.Recall(),tf.keras.metrics.TruePositives(), tf.keras.metrics.TrueNegatives(), tf.keras.metrics.FalsePositives(), tf.keras.metrics.FalseNegatives()])
precision = model_.evaluate(x_test_data,y_test,batch_size = 128)
print('Our F1-score score is {} - Test group.'.format(precision[2]))

Complete Training

In [ ]:
# Read image and resize using INTER_AREA

x_train_validation_data = list()
x_test_data = list()

for number in tqdm(range(0,2451)):
    im = cv2.imread('../input/images-grayscale/X_train_validation_png/image0' + str(number) + '.png',cv2.IMREAD_GRAYSCALE)
    im = cv2.resize(im, (256,256), interpolation = cv2.INTER_AREA)
    x_train_validation_data.append(im)
    
#Loading the y data
y_train_validation = np.load('../input/brain-tumor-dataset-modified/y_train_validation.npy', allow_pickle = True)
y_test = np.load('../input/brain-tumor-dataset-modified/y_test.npy', allow_pickle = True)

In [ ]:
# Activating the TPU

tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
for number in tqdm(range(0,613)):
    im = cv2.imread('../input/images-grayscale/X_test_png/image0' + str(number) + '.png',cv2.IMREAD_GRAYSCALE)
    im = cv2.resize(im, (256,256), interpolation = cv2.INTER_AREA)
    #_, im = FFT_log(im)
    #_, im = NEX(im,0.41)
    #_, im = cut_sup_inf(im, 0.9)
    #_, im = low_pass(im, 50)
    #_, im = noise(im, 1)
    x_test_data.append(im)

In [ ]:
from tensorflow.keras import backend as K

def custom_f1(y_true, y_pred):    
    def recall_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        
        recall = TP / (Positives+K.epsilon())    
        return recall 
    
    
    def precision_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    
        precision = TP / (Pred_Positives+K.epsilon())
        return precision 
    
    precision, recall = precision_m(y_true, y_pred), recall_m(y_true, y_pred)
    
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
#Reshape the images

x_train_validation_data = (np.array(x_train_validation_data).reshape((len(x_train_validation_data), 256, 256, 1))).astype('float16') / 255
x_test_data = (np.array(x_test_data).reshape((len(x_test_data), 256, 256, 1))).astype('float16') / 255

In [ ]:
start_time = datetime.datetime.now()
resultados = []


SIZE = len(x_train_validation_data[0])
seresnext50, _ = Classifiers.get('mobilenet')

input_tensor = keras.layers.Input(shape=(SIZE, SIZE,1))
x = keras.layers.Conv2D(3,(1,1),padding='same')(input_tensor)
base_model = seresnext50(input_shape = (SIZE, SIZE,3), include_top = False, weights = 'imagenet')(x)
y = keras.layers.GlobalAveragePooling2D()(base_model)
y2 = tf.keras.layers.BatchNormalization()(y)
y3 = tf.keras.layers.Flatten()(y2)
y4 = tf.keras.layers.Dense(1026, activation="relu")(y3)
y5 = tf.keras.layers.Dropout(0.5)(y4)
output = keras.layers.Dense(3, activation='softmax')(y5)
model = keras.models.Model(inputs=[input_tensor], outputs=[output])
model.summary()

model.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=[tf.keras.metrics.AUC(multi_label=True), custom_f1,'accuracy',
                                                                         tf.keras.metrics.Precision(), keras.metrics.Recall(),tf.keras.metrics.TruePositives(), tf.keras.metrics.TrueNegatives(), tf.keras.metrics.FalsePositives(), tf.keras.metrics.FalseNegatives()])


name= 'MRI_mobilenet.h5'
rlr = ReduceLROnPlateau(monitor = 'val_custom_f1', factor = 0.1, patience = 3, verbose = 1, min_delta = 1e-4, min_lr = 1e-6, mode = 'max', cooldown=1)
ckp = ModelCheckpoint(name,monitor = 'val_custom_f1', verbose = 1, save_best_only = True, mode = 'max')    
es = EarlyStopping(monitor = 'val_custom_f1', min_delta = 1e-4, patience =10, mode = 'max', restore_best_weights = True, verbose = 1)
history = model.fit(x_train_validation_data, y_train_validation,
                    batch_size = 32, epochs = 100,
                    validation_data = (x_test_data, y_test),
                    callbacks=[rlr,es,ckp])

end_time = datetime.datetime.now()
print('Duration: {}'.format(end_time - start_time))

In [ ]:
dependencies = {'custom_f1': custom_f1}
model_= tf.keras.models.load_model('./MRI_mobilenet.h5', custom_objects=dependencies)
model_.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=[tf.keras.metrics.AUC(multi_label=True), custom_f1,'accuracy',
                                                                          tf.keras.metrics.Precision(), keras.metrics.Recall(),tf.keras.metrics.TruePositives(), tf.keras.metrics.TrueNegatives(), tf.keras.metrics.FalsePositives(), tf.keras.metrics.FalseNegatives()])

In [ ]:
precision = model_.evaluate(x_test_data,
                            y_test,
                            batch_size = 32)
resultados.append(precision)

In [ ]:
resultados

Let's avaliate our networks in some conditions

In [ ]:
from tensorflow.keras import backend as K

def custom_f1(y_true, y_pred):    
    def recall_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        
        recall = TP / (Positives+K.epsilon())    
        return recall 
    
    
    def precision_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    
        precision = TP / (Pred_Positives+K.epsilon())
        return precision 
    
    precision, recall = precision_m(y_true, y_pred), recall_m(y_true, y_pred)
    
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def NEX(np_pixel_array, percentage):
  percentage_zeros = percentage
  propor = percentage_zeros 
  siz = np_pixel_array.shape[0] - (np_pixel_array.shape[0]*propor)


  im = np_pixel_array
  FFT_img = np.fft.fft2(im)
  FFT_img_shift = np.fft.fftshift(FFT_img)

  real = FFT_img_shift.real
  phases = FFT_img_shift.imag

  real[int(siz):,:] = 0 #Ex.: P/ baixo de certo número é 0
  real_mod = real

  phases[int(siz):,:] = 0 #Ex.: P/ baixo de certo número é 0

  FFT_img_abs = np.abs(FFT_img)
  FFT_img_shift_log = 10*np.log(np.fft.fftshift(FFT_img_abs))
  FFT_img_shift_log[int(siz):,:] = 0

  FFT_img_shift_mod = np.empty(real.shape, dtype=complex)

  FFT_img_shift_mod.real = real_mod
  FFT_img_shift_mod.imag = phases

  FFT_img_mod = np.fft.ifftshift(FFT_img_shift_mod)
  img_mod = np.fft.ifft2(FFT_img_mod)
  img_mod = np.abs(img_mod)
  return FFT_img_shift_log, img_mod

def cut_sup_inf(np_pixel_array, percentage):
  im = np_pixel_array
  FFT_img = np.fft.fft2(im)
  FFT_img_shift = np.fft.fftshift(FFT_img)
  FFT_img_shift_zeros = np.zeros(FFT_img_shift.shape)

  percentage_zeros = percentage
  percentage_data = 1 - percentage_zeros
  percentage_rows = int(FFT_img_shift.shape[0]*percentage_data)
  sup_row = int((FFT_img_shift.shape[0]/2)-(percentage_rows/2))
  inf_row = int((FFT_img_shift.shape[0]/2)+(percentage_rows/2))

  for i in range(0,int(FFT_img_shift.shape[0])):
    for j in range(0,int(FFT_img_shift.shape[1])):
      if (i >= sup_row) and (i <= inf_row):
        FFT_img_shift_zeros[i][j] = 1

  mask = FFT_img_shift_zeros
  FFT_img_shift_mask = FFT_img_shift*mask
  FFT_img_shift_mask_visu = 10*np.log(np.fft.fftshift(np.abs(FFT_img)))*mask

  FFT_img_inv = np.fft.ifftshift(FFT_img_shift_mask)
  img_inv = np.fft.ifft2(FFT_img_inv)
  img_mod = np.abs(img_inv)
  return FFT_img_shift_mask_visu, img_mod

def low_pass(np_pixel_array, r):
  im = np_pixel_array
  FFT_img = np.fft.fft2(im)
  FFT_img_shift = np.fft.fftshift(FFT_img)
  FFT_img_shift_zeros = np.zeros(FFT_img_shift.shape)

  #r = 1 # how narrower the window is (maior pior)
  ham = np.hamming(FFT_img_shift.shape[0])[:,None] # 1D hamming
  ham2d = np.sqrt(np.dot(ham, ham.T)) ** r # expand to 2D hamming

  mask = FFT_img_shift_zeros
  FFT_img_shift_mask = FFT_img_shift*ham2d 
  FFT_img_shift_log = 10*np.log(np.fft.fftshift(np.abs(FFT_img)))*ham2d 

  FFT_img_inv = np.fft.ifftshift(FFT_img_shift_mask)
  img_inv = np.fft.ifft2(FFT_img_inv)
  img_mod = np.abs(img_inv)
  return FFT_img_shift_log, img_mod


def noise(np_pixel_array, sigma):
  img = np_pixel_array
  gauss = img + np.random.normal(0,sigma,img.shape)
  img_gauss = np.clip(gauss, 0, 255)
  return img_gauss

In [ ]:
resultados = []

n = 1 #1, 1,100 ,10
jump = 0.1 # 0.1, 0.1, 10, 1
y_test = np.load('../input/brain-tumor-dataset-modified/y_test.npy', allow_pickle = True)

for i in tqdm(range(0,10,1)):
    if jump == 0.1:
        j = i/10
    if jump == 10:
        j = i*10
    if jump == 1:
        j = i
    x_test_data = []
    for number in range(0,613):
        im = cv2.imread('../input/images-grayscale/X_test_png/image0' + str(number) + '.png',cv2.IMREAD_GRAYSCALE)
        im = cv2.resize(im, (256,256), interpolation = cv2.INTER_AREA)
        _, im = NEX(im,j) # mask percentage
        #_, im = cut_sup_inf(im, j) # mask percentage
        #_, im = low_pass(im, j) #size of window
        #im = noise(im, j) # sigma
        x_test_data.append(im)
    x_test_data = (np.array(x_test_data).reshape((len(x_test_data), 256, 256, 1))).astype('float16') / 255
    dependencies = {'custom_f1': custom_f1}
    model_= tf.keras.models.load_model('../input/classification-networks/MRI_mobilenet_class.h5', custom_objects=dependencies)
    model_.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=[tf.keras.metrics.AUC(multi_label=True), custom_f1,'accuracy',
                                                                              tf.keras.metrics.Precision(), keras.metrics.Recall(),tf.keras.metrics.TruePositives(), tf.keras.metrics.TrueNegatives(), tf.keras.metrics.FalsePositives(), tf.keras.metrics.FalseNegatives()])
    precision = model_.evaluate(x_test_data,y_test,batch_size = 32)
    resultados.append(precision)

In [ ]:
resultados

In [ ]:
resultados = np.array(resultados)
matrix={}
matrix['loss']= resultados[:,0]
matrix['AUC']= resultados[:,1]
matrix['F1']= resultados[:,2]
matrix['Acc']= resultados[:,3]
matrix['Prec']= resultados[:,4]
matrix['Rec']= resultados[:,5]
results = pd.DataFrame(matrix)
results

In [ ]:
results.to_excel("NEX_Mobilenet_class.xlsx")  

-----------------------------------------------------------------------------------

In [ ]:
resultados = []

n = 1 #1, 1,100 ,10
jump = 0.1 # 0.1, 0.1, 10, 1
y_test = np.load('../input/brain-tumor-dataset-modified/y_test.npy', allow_pickle = True)

for i in tqdm(range(0,10,1)):
    if jump == 0.1:
        j = i/10
    if jump == 10:
        j = i*10
    if jump == 1:
        j = i
    x_test_data = []
    for number in range(0,613):
        im = cv2.imread('../input/images-grayscale/X_test_png/image0' + str(number) + '.png',cv2.IMREAD_GRAYSCALE)
        im = cv2.resize(im, (256,256), interpolation = cv2.INTER_AREA)
        #_, im = NEX(im,j) # mask percentage
        _, im = cut_sup_inf(im, j) # mask percentage
        #_, im = low_pass(im, j) #size of window
        #im = noise(im, j) # sigma
        x_test_data.append(im)
    x_test_data = (np.array(x_test_data).reshape((len(x_test_data), 256, 256, 1))).astype('float16') / 255
    dependencies = {'custom_f1': custom_f1}
    model_= tf.keras.models.load_model('../input/classification-networks/MRI_mobilenet_class.h5', custom_objects=dependencies)
    model_.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=[tf.keras.metrics.AUC(multi_label=True), custom_f1,'accuracy',
                                                                              tf.keras.metrics.Precision(), keras.metrics.Recall(),tf.keras.metrics.TruePositives(), tf.keras.metrics.TrueNegatives(), tf.keras.metrics.FalsePositives(), tf.keras.metrics.FalseNegatives()])
    precision = model_.evaluate(x_test_data,y_test,batch_size = 32)
    resultados.append(precision)

resultados = np.array(resultados)
matrix={}
matrix['loss']= resultados[:,0]
matrix['AUC']= resultados[:,1]
matrix['F1']= resultados[:,2]
matrix['Acc']= resultados[:,3]
matrix['Prec']= resultados[:,4]
matrix['Rec']= resultados[:,5]
results = pd.DataFrame(matrix)

results.to_excel("CUT_SUP_INF_Mobilenet_class.xlsx")  

-----------------------------------------------------------------------------

In [ ]:
resultados = []

n = 100 #1, 1,100 ,10
jump = 10 # 0.1, 0.1, 10, 1
y_test = np.load('../input/brain-tumor-dataset-modified/y_test.npy', allow_pickle = True)

for i in tqdm(range(0,10,1)):
    if jump == 0.1:
        j = i/10
    if jump == 10:
        j = i*10
    if jump == 1:
        j = i
    x_test_data = []
    for number in range(0,613):
        im = cv2.imread('../input/images-grayscale/X_test_png/image0' + str(number) + '.png',cv2.IMREAD_GRAYSCALE)
        im = cv2.resize(im, (256,256), interpolation = cv2.INTER_AREA)
        #_, im = NEX(im,j) # mask percentage
        #_, im = cut_sup_inf(im, j) # mask percentage
        _, im = low_pass(im, j) #size of window
        #im = noise(im, j) # sigma
        x_test_data.append(im)
    x_test_data = (np.array(x_test_data).reshape((len(x_test_data), 256, 256, 1))).astype('float16') / 255
    dependencies = {'custom_f1': custom_f1}
    model_= tf.keras.models.load_model('../input/classification-networks/MRI_mobilenet_class.h5', custom_objects=dependencies)
    model_.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=[tf.keras.metrics.AUC(multi_label=True), custom_f1,'accuracy',
                                                                              tf.keras.metrics.Precision(), keras.metrics.Recall(),tf.keras.metrics.TruePositives(), tf.keras.metrics.TrueNegatives(), tf.keras.metrics.FalsePositives(), tf.keras.metrics.FalseNegatives()])
    precision = model_.evaluate(x_test_data,y_test,batch_size = 32)
    resultados.append(precision)

resultados = np.array(resultados)
matrix={}
matrix['loss']= resultados[:,0]
matrix['AUC']= resultados[:,1]
matrix['F1']= resultados[:,2]
matrix['Acc']= resultados[:,3]
matrix['Prec']= resultados[:,4]
matrix['Rec']= resultados[:,5]
results = pd.DataFrame(matrix)

results.to_excel("LOWPASS_Mobilenet_class.xlsx")  

----------------------------------------------------------------------------------------------------

In [ ]:
resultados = []

n = 100 #1, 1,100 ,100
jump = 10 # 0.1, 0.1, 10, 10
y_test = np.load('../input/brain-tumor-dataset-modified/y_test.npy', allow_pickle = True)

for i in tqdm(range(0,10,1)):
    if jump == 0.1:
        j = i/10
    if jump == 10:
        j = i*10
    if jump == 1:
        j = i
    x_test_data = []
    for number in range(0,613):
        im = cv2.imread('../input/images-grayscale/X_test_png/image0' + str(number) + '.png',cv2.IMREAD_GRAYSCALE)
        im = cv2.resize(im, (256,256), interpolation = cv2.INTER_AREA)
        #_, im = NEX(im,j) # mask percentage
        #_, im = cut_sup_inf(im, j) # mask percentage
        #_, im = low_pass(im, j) #size of window
        im = noise(im, j) # sigma
        x_test_data.append(im)
    x_test_data = (np.array(x_test_data).reshape((len(x_test_data), 256, 256, 1))).astype('float16') / 255
    dependencies = {'custom_f1': custom_f1}
    model_= tf.keras.models.load_model('../input/classification-networks/MRI_mobilenet_class.h5', custom_objects=dependencies)
    model_.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=[tf.keras.metrics.AUC(multi_label=True), custom_f1,'accuracy',
                                                                              tf.keras.metrics.Precision(), keras.metrics.Recall(),tf.keras.metrics.TruePositives(), tf.keras.metrics.TrueNegatives(), tf.keras.metrics.FalsePositives(), tf.keras.metrics.FalseNegatives()])
    precision = model_.evaluate(x_test_data,y_test,batch_size = 32)
    resultados.append(precision)

resultados = np.array(resultados)
matrix={}
matrix['loss']= resultados[:,0]
matrix['AUC']= resultados[:,1]
matrix['F1']= resultados[:,2]
matrix['Acc']= resultados[:,3]
matrix['Prec']= resultados[:,4]
matrix['Rec']= resultados[:,5]
results = pd.DataFrame(matrix)

results.to_excel("NOISE_Mobilenet_class.xlsx")  

# 6. Trying classify a image

In [ ]:
img = cv2.imread('../input/images-grayscale/X_test_png/image0227.png',cv2.IMREAD_GRAYSCALE)
#img = cv2.equalizeHist(img)
img = cv2.resize(img, (256,256), interpolation = cv2.INTER_AREA)
img = (np.array(img).reshape((1, 256, 256, 1))).astype('float16') / 255
dependencies = {'custom_f1': custom_f1}
model_= tf.keras.models.load_model('../input/classification-networks/MRI_seresnext50_class.h5', custom_objects=dependencies)
prediction = model_.predict(img)
print(prediction)

if (prediction[0][0] >  prediction[0][1]) and (prediction[0][0] >  prediction[0][2]):
    print("Meningioma")
if (prediction[0][1] >  prediction[0][0]) and (prediction[0][1] >  prediction[0][2]):
    print("Glioma")
if (prediction[0][2] >  prediction[0][0]) and (prediction[0][2] >  prediction[0][1]):
    print("Pituitary tumor")

In [ ]:
plt.figsize=(8, 6)
u = cv2.imread('../input/teste2/IMG-0003-00075.png',cv2.IMREAD_GRAYSCALE)
plt.imshow(u, cmap='gray')

# 7. Initializing brain tumor segmentation

In [ ]:
# Loading the saved data for classification - PNG method

#os.makedirs("./X_train_validation_png")
#os.makedirs("./X_test_png")
#os.makedirs("./y_train_validation_png")
#os.makedirs("./y_test_png")


X_train_validation_jpg = np.load('./X_train_validation.npy', allow_pickle = True)
for i in tqdm(range(0,2451)):    
    X1 = cv2.convertScaleAbs(X_train_validation_jpg[i], alpha = 0.03).tolist()
    plt.imsave('./X_train_validation_png/image0' + str(i) + '.png', X1, cmap='gray')
    

X_test_jpg = np.load('./X_test.npy', allow_pickle = True)
for i in tqdm(range(0,613)):    
    X2 = cv2.convertScaleAbs(X_test_jpg[i], alpha = 0.03).tolist()
    plt.imsave('./X_test_png/image0' + str(i) + '.png', X2, cmap='gray')
    
y_train_validation_jpg = np.load('./y_train_validation.npy', allow_pickle = True)
for i in tqdm(range(0,2451)):    
    X1 = cv2.convertScaleAbs(y_train_validation_jpg[i], alpha = 1).tolist()
    plt.imsave('./y_train_validation_png/image0' + str(i) + '.png', X1, cmap='gray')
    

y_test_jpg = np.load('./y_test.npy', allow_pickle = True)
for i in tqdm(range(0,613)):    
    X2 = cv2.convertScaleAbs(y_test_jpg[i], alpha = 1).tolist()
    plt.imsave('./y_test_png/image0' + str(i) + '.png', X2, cmap='gray')
    

In [ ]:
# Save zipfile of X_train_validation

import os
import zipfile
 
fantasy_zip = zipfile.ZipFile('./X_train_validation_png.zip', 'w')
 
for folder, subfolders, files in os.walk('./X_train_validation_png'):
 
    for file in files:
        if file.endswith('.png'):
            fantasy_zip.write(os.path.join(folder, file), os.path.relpath(os.path.join(folder,file), './PNEUMOTHORAX'), compress_type = zipfile.ZIP_DEFLATED)
 
fantasy_zip.close()

In [ ]:
# Save zipfile of X_test

import os
import zipfile
 
fantasy_zip = zipfile.ZipFile('./X_test_png.zip', 'w')
 
for folder, subfolders, files in os.walk('./X_test_png'):
 
    for file in files:
        if file.endswith('.png'):
            fantasy_zip.write(os.path.join(folder, file), os.path.relpath(os.path.join(folder,file), './PNEUMOTHORAX'), compress_type = zipfile.ZIP_DEFLATED)
 
fantasy_zip.close()

In [ ]:
# Save zipfile of y_train_validation

import os
import zipfile
 
fantasy_zip = zipfile.ZipFile('./y_train_validation_png.zip', 'w')
 
for folder, subfolders, files in os.walk('./y_train_validation_png'):
 
    for file in files:
        if file.endswith('.png'):
            fantasy_zip.write(os.path.join(folder, file), os.path.relpath(os.path.join(folder,file), './PNEUMOTHORAX'), compress_type = zipfile.ZIP_DEFLATED)
 
fantasy_zip.close()

In [ ]:
# Save zipfile of y_test

import os
import zipfile
 
fantasy_zip = zipfile.ZipFile('./y_test_png.zip', 'w')
 
for folder, subfolders, files in os.walk('./y_test_png'):
 
    for file in files:
        if file.endswith('.png'):
            fantasy_zip.write(os.path.join(folder, file), os.path.relpath(os.path.join(folder,file), './PNEUMOTHORAX'), compress_type = zipfile.ZIP_DEFLATED)
 
fantasy_zip.close()

Good point to restart

In [ ]:
# Read image and resize using INTER_AREA
x_train_validation_data = list()
x_test_data = list()
y_train_validation_data = list()
y_test_data = list()
X_train = list()
Y_train = list()

for number in tqdm(range(0,2451)):
    image = cv2.imread('../input/images-masks/X_train_validation_png/image0' + str(number) + '.png',0)
    mask = cv2.imread('../input/images-masks/y_train_validation_png/image0' + str(number) + '.png',0)
    #X_train.append(image)
    #Y_train.append(mask)
    image = cv2.resize(image, (256,256), interpolation = cv2.INTER_AREA)
    mask = cv2.resize(mask, (256,256), interpolation = cv2.INTER_AREA)
    X_train.append(image)
    Y_train.append(mask)
    x_train_validation_data.append(image)
    y_train_validation_data.append(mask)
    
for number in tqdm(range(0,613)):
    im = cv2.imread('../input/images-masks/X_test_png/image0' + str(number) + '.png',0)
    im = cv2.resize(im, (256,256), interpolation = cv2.INTER_AREA)
    x_test_data.append(im)
for number in tqdm(range(0,613)):
    im = cv2.imread('../input/images-masks/y_test_png/image0' + str(number) + '.png',0)
    im = cv2.resize(im, (256,256), interpolation = cv2.INTER_AREA)
    y_test_data.append(im)

In [ ]:
#transform = A.Compose([A.Rotate(limit=3),
#                       A.RandomScale(scale_limit=0.3),
#                       A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.3, rotate_limit=3),
#                       A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3),
#                       A.GridDistortion(num_steps=5, distort_limit=0.2),
#                       A.ElasticTransform(p=0.2)])

In [ ]:
#for number in tqdm(range(0,1049)):
#    image = cv2.imread('../input/images-masks/X_train_validation_png/image0' + str(number) + '.png',cv2.IMREAD_COLOR)
#    mask = cv2.imread('../input/images-masks/y_train_validation_png/image0' + str(number) + '.png',0)
#    
#    transformed = transform(image= image, mask = mask)
#    transformed_image = transformed['image']
#    transformed_mask = transformed['mask']  
#    
#    image = cv2.resize(transformed_image, (256,256), interpolation = cv2.INTER_AREA)
#    mask = cv2.resize(transformed_mask, (256,256), interpolation = cv2.INTER_AREA)
#    
#    x_train_validation_data.append(image)
#    y_train_validation_data.append(mask)

In [ ]:
''''   
for number in tqdm(range(0,2451)):
    image = cv2.imread('../input/images-masks/X_train_validation_png/image0' + str(number) + '.png',cv2.IMREAD_COLOR)
    mask = cv2.imread('../input/images-masks/y_train_validation_png/image0' + str(number) + '.png',0)
    image = cv2.resize(transformed_image, (256,256), interpolation = cv2.INTER_AREA)
    mask = cv2.resize(transformed_mask, (256,256), interpolation = cv2.INTER_AREA)
    x_train_validation_data.append(image)
    y_train_validation_data.append(mask)
''''  

In [ ]:
#Show images

f, axarr = plt.subplots(2,2)
axarr[0,0].imshow(x_train_validation_data[400], cmap='gray')
axarr[0,1].imshow(y_train_validation_data[400], cmap='gray')
axarr[1,0].imshow(x_train_validation_data[600], cmap='gray')
axarr[1,1].imshow(y_train_validation_data[600], cmap='gray')

In [ ]:
#Reshape the images

x_train_validation_data = np.array(x_train_validation_data).reshape((len(x_train_validation_data), 256, 256, 1)).astype('float16') / 255
x_test_data = np.array(x_test_data).reshape((len(x_test_data), 256, 256, 1)).astype('float16') / 255
y_train_validation_data = np.array(y_train_validation_data).reshape((len(y_train_validation_data), 256, 256, 1)).astype('float16') / 255
y_test_data = np.array(y_test_data).reshape((len(y_test_data), 256, 256, 1)).astype('float16') / 255

In [ ]:
""""
BACKBONE = 'vgg16'
BATCH_SIZE = 16
LR = 0.0001
EPOCHS = 40
n_classes = 1
activation = 'sigmoid'
model = sm.Unet(BACKBONE, classes=n_classes, activation=activation)
optim = keras.optimizers.Adam(LR)
dice_loss = sm.losses.DiceLoss()
focal_loss = sm.losses.BinaryFocalLoss()
total_loss = dice_loss + (1 * focal_loss)
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]
model.compile(optim, total_loss, metrics)
history=model.fit(x_train_validation_data, y_train_validation_data, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1, validation_data=(x_test_data, y_test_data))
""""

In [ ]:
images = []
masks = []

transform = A.Compose([A.Rotate(limit=3),
                       A.RandomScale(scale_limit=0.3),
                       A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.3, rotate_limit=3),
                       A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3),
                       A.GridDistortion(num_steps=5, distort_limit=0.2),
                       A.ElasticTransform(p=0.2)])

for image, mask in tqdm(zip(X_train, Y_train)):
        transformed = transform(image= image, mask = mask)
        transformed_image = transformed['image']
        transformed_mask = transformed['mask']
        image = cv2.resize(transformed_image, (256,256), interpolation = cv2.INTER_AREA)
        mask = cv2.resize(transformed_mask, (256,256), interpolation = cv2.INTER_AREA)
        images.append(image)
        masks.append(mask)

x_train_images = np.array(images).reshape((len(images), 256, 256, 1)).astype('float16') / 255
y_train_masks = np.array(masks).reshape((len(masks), 256, 256, 1)).astype('float16') / 255  

In [ ]:
# Deleting useless data

del X_train
del Y_train
del images
del masks
gc.collect()

In [ ]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
# Constructing the NN and executing the tran and validation
start_time = datetime.datetime.now()

kfold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)

resultados = []
j = 0

for train_index, test_index in kfold.split(x_train_validation_data,np.zeros(shape = (x_train_validation_data.shape[0],1))):
    
    SIZE = len(x_train_validation_data[0])          
    
    x_train_images_ = np.append(x_train_validation_data[train_index],x_train_images[train_index][:len(x_train_images[train_index])//2], axis = 0)
    y_train_masks_ = np.append(y_train_validation_data[train_index],y_train_masks[train_index][:len(y_train_masks[train_index])//2], axis = 0)
    
    with tpu_strategy.scope():
        BACKBONE = 'seresnext50'
        BATCH_SIZE = 8 * tpu_strategy.num_replicas_in_sync
        LR = 0.0001
        EPOCHS = 100
        n_classes = 1
        activation = 'sigmoid'
        base_model = sm.Unet(BACKBONE, classes=n_classes, activation=activation)

        inp = keras.layers.Input(shape=(None, None, 1))
        l1 = keras.layers.Conv2D(3, (1, 1))(inp)
        out = base_model(l1)
        model = keras.models.Model(inp, out, name=base_model.name)

        optim = keras.optimizers.Adam(learning_rate=LR)
        dice_loss = sm.losses.DiceLoss()
        #focal_loss = sm.losses.BinaryFocalLoss()
        #total_loss = dice_loss + (1 * focal_loss)
        metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5), "accuracy"]
        model.compile(optim, dice_loss, metrics)
        model.summary()

        if j == 0:
            k = 'val_iou_score'
        else:
            k = 'val_iou_score'#_'+str(j)

        name= 'MRI_seresnext50.h5'
        rlr = ReduceLROnPlateau(monitor = k, factor = 0.1, patience = 6, verbose = 1, min_delta = 1e-4, min_lr = 1e-5, mode = 'max', cooldown=1)
        ckp = ModelCheckpoint(name,monitor = k,verbose = 1, save_best_only = True, mode = 'max')    
        es = EarlyStopping(monitor = k, min_delta = 1e-4, patience =12, mode = 'max', restore_best_weights = True, verbose = 1)
        history = model.fit(x_train_images_, y_train_masks_,
                            batch_size = BATCH_SIZE, epochs = EPOCHS, 
                            validation_data = (x_train_validation_data[test_index], y_train_validation_data[test_index]),
                            callbacks=[rlr, es,ckp])

        dependencies = {'dice_loss': dice_loss,'iou_score': sm.metrics.IOUScore(threshold=0.5), 'f1-score': sm.metrics.FScore(threshold=0.5)}
        model_= tf.keras.models.load_model('./MRI_seresnext50.h5', custom_objects=dependencies)
        model_.compile(loss=dice_loss, optimizer=optim, metrics=metrics)
        precision = model_.evaluate(x_train_validation_data[test_index],
                                    y_train_validation_data[test_index],
                                    batch_size = BATCH_SIZE)


        resultados.append(precision)
        j += 1
        
end_time = datetime.datetime.now()
print('Duration: {}'.format(end_time - start_time))

In [ ]:
resultados

**Treinar modelo**

In [ ]:
# Read image and resize using INTER_AREA
x_train_validation_data = list()
x_test_data = list()
y_train_validation_data = list()
y_test_data = list()
X_train = list()
Y_train = list()

for number in tqdm(range(0,2451)):
    image = cv2.imread('../input/images-masks/X_train_validation_png/image0' + str(number) + '.png',0)
    mask = cv2.imread('../input/images-masks/y_train_validation_png/image0' + str(number) + '.png',0)
    image = cv2.resize(image, (256,256), interpolation = cv2.INTER_AREA)
    mask = cv2.resize(mask, (256,256), interpolation = cv2.INTER_AREA)
    X_train.append(image)
    Y_train.append(mask)
    x_train_validation_data.append(image)
    y_train_validation_data.append(mask)
        
for number in tqdm(range(0,613)):
    im = cv2.imread('../input/images-masks/y_test_png/image0' + str(number) + '.png',0)
    im = cv2.resize(im, (256,256), interpolation = cv2.INTER_AREA)
    y_test_data.append(im)

In [ ]:
#Reshape the images

x_train_validation_data = np.array(x_train_validation_data).reshape((len(x_train_validation_data), 256, 256, 1)).astype('float16') / 255
y_train_validation_data = np.array(y_train_validation_data).reshape((len(y_train_validation_data), 256, 256, 1)).astype('float16') / 255
y_test_data = np.array(y_test_data).reshape((len(y_test_data), 256, 256, 1)).astype('float16') / 255

In [ ]:
images = []
masks = []

transform = A.Compose([A.Rotate(limit=3),
                       A.RandomScale(scale_limit=0.3),
                       A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.3, rotate_limit=3),
                       A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3),
                       A.GridDistortion(num_steps=5, distort_limit=0.2),
                       A.ElasticTransform(p=0.2)])

for image, mask in tqdm(zip(X_train, Y_train)):
        transformed = transform(image= image, mask = mask)
        transformed_image = transformed['image']
        transformed_mask = transformed['mask']
        image = cv2.resize(transformed_image, (256,256), interpolation = cv2.INTER_AREA)
        mask = cv2.resize(transformed_mask, (256,256), interpolation = cv2.INTER_AREA)
        images.append(image)
        masks.append(mask)

x_train_images = np.array(images).reshape((len(images), 256, 256, 1)).astype('float16') / 255
y_train_masks = np.array(masks).reshape((len(masks), 256, 256, 1)).astype('float16') / 255  

for number in tqdm(range(0,613)):
    im = cv2.imread('../input/images-masks/X_test_png/image0' + str(number) + '.png',0)
    im = cv2.resize(im, (256,256), interpolation = cv2.INTER_AREA)
    #_, im = NEX(im,j) # mask percentage
    #_, im = cut_sup_inf(im, j) # mask percentage
    #_, im = low_pass(im, j) #size of window
    #im = noise(im, j) # sigma
    x_test_data.append(im)

In [ ]:
x_test_data = np.array(x_test_data).reshape((len(x_test_data), 256, 256, 1)).astype('float16') / 255

In [ ]:
# Constructing the NN and executing the tran and validation
start_time = datetime.datetime.now()

kfold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
resultados = []

SIZE = len(x_train_validation_data[0])          

x_train_images_ = np.append(x_train_validation_data,x_train_images[:len(x_train_images)//2], axis = 0)
y_train_masks_ = np.append(y_train_validation_data,y_train_masks[:len(y_train_masks)//2], axis = 0)


BACKBONE = 'mobilenet'
BATCH_SIZE = 32
LR = 0.0001
EPOCHS = 100
n_classes = 1
activation = 'sigmoid'
base_model = sm.Unet(BACKBONE, classes=n_classes, activation=activation)

inp = keras.layers.Input(shape=(None, None, 1))
l1 = keras.layers.Conv2D(3, (1, 1))(inp)
out = base_model(l1)
model = keras.models.Model(inp, out, name=base_model.name)

optim = keras.optimizers.Adam(learning_rate=LR)
dice_loss = sm.losses.DiceLoss()
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5), "accuracy"]
model.compile(optim, dice_loss, metrics)
model.summary()

k = 'val_iou_score'

name= 'MRI_mobilenet.h5'
rlr = ReduceLROnPlateau(monitor = k, factor = 0.1, patience = 6, verbose = 1, min_delta = 1e-4, min_lr = 1e-5, mode = 'max', cooldown=1)
ckp = ModelCheckpoint(name,monitor = k,verbose = 1, save_best_only = True, mode = 'max')    
es = EarlyStopping(monitor = k, min_delta = 1e-4, patience =12, mode = 'max', restore_best_weights = True, verbose = 1)
history = model.fit(x_train_images_, y_train_masks_,
                    batch_size = BATCH_SIZE, epochs = EPOCHS, 
                    validation_data = (x_test_data, y_test_data),
                    callbacks=[rlr,es,ckp])


end_time = datetime.datetime.now()
print('Duration: {}'.format(end_time - start_time))

Tá terminando

In [ ]:
y_test_data = []

for number in tqdm(range(0,613)):
    im = cv2.imread('../input/images-masks/y_test_png/image0' + str(number) + '.png',0)
    im = cv2.resize(im, (256,256), interpolation = cv2.INTER_AREA)
    y_test_data.append(im)
    
y_test_data = np.array(y_test_data).reshape((len(y_test_data), 256, 256, 1)).astype('float16') / 255

In [ ]:
from tensorflow.keras import backend as K

def custom_f1(y_true, y_pred):    
    def recall_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        
        recall = TP / (Positives+K.epsilon())    
        return recall 
    
    
    def precision_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    
        precision = TP / (Pred_Positives+K.epsilon())
        return precision 
    
    precision, recall = precision_m(y_true, y_pred), recall_m(y_true, y_pred)
    
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def NEX(np_pixel_array, percentage):
  percentage_zeros = percentage
  propor = percentage_zeros 
  siz = np_pixel_array.shape[0] - (np_pixel_array.shape[0]*propor)


  im = np_pixel_array
  FFT_img = np.fft.fft2(im)
  FFT_img_shift = np.fft.fftshift(FFT_img)

  real = FFT_img_shift.real
  phases = FFT_img_shift.imag

  real[int(siz):,:] = 0 #Ex.: P/ baixo de certo número é 0
  real_mod = real

  phases[int(siz):,:] = 0 #Ex.: P/ baixo de certo número é 0

  FFT_img_abs = np.abs(FFT_img)
  FFT_img_shift_log = 10*np.log(np.fft.fftshift(FFT_img_abs))
  FFT_img_shift_log[int(siz):,:] = 0

  FFT_img_shift_mod = np.empty(real.shape, dtype=complex)

  FFT_img_shift_mod.real = real_mod
  FFT_img_shift_mod.imag = phases

  FFT_img_mod = np.fft.ifftshift(FFT_img_shift_mod)
  img_mod = np.fft.ifft2(FFT_img_mod)
  img_mod = np.abs(img_mod)
  return FFT_img_shift_log, img_mod

def cut_sup_inf(np_pixel_array, percentage):
  im = np_pixel_array
  FFT_img = np.fft.fft2(im)
  FFT_img_shift = np.fft.fftshift(FFT_img)
  FFT_img_shift_zeros = np.zeros(FFT_img_shift.shape)

  percentage_zeros = percentage
  percentage_data = 1 - percentage_zeros
  percentage_rows = int(FFT_img_shift.shape[0]*percentage_data)
  sup_row = int((FFT_img_shift.shape[0]/2)-(percentage_rows/2))
  inf_row = int((FFT_img_shift.shape[0]/2)+(percentage_rows/2))

  for i in range(0,int(FFT_img_shift.shape[0])):
    for j in range(0,int(FFT_img_shift.shape[1])):
      if (i >= sup_row) and (i <= inf_row):
        FFT_img_shift_zeros[i][j] = 1

  mask = FFT_img_shift_zeros
  FFT_img_shift_mask = FFT_img_shift*mask
  FFT_img_shift_mask_visu = 10*np.log(np.fft.fftshift(np.abs(FFT_img)))*mask

  FFT_img_inv = np.fft.ifftshift(FFT_img_shift_mask)
  img_inv = np.fft.ifft2(FFT_img_inv)
  img_mod = np.abs(img_inv)
  return FFT_img_shift_mask_visu, img_mod

def low_pass(np_pixel_array, r):
  im = np_pixel_array
  FFT_img = np.fft.fft2(im)
  FFT_img_shift = np.fft.fftshift(FFT_img)
  FFT_img_shift_zeros = np.zeros(FFT_img_shift.shape)

  #r = 1 # how narrower the window is (maior pior)
  ham = np.hamming(FFT_img_shift.shape[0])[:,None] # 1D hamming
  ham2d = np.sqrt(np.dot(ham, ham.T)) ** r # expand to 2D hamming

  mask = FFT_img_shift_zeros
  FFT_img_shift_mask = FFT_img_shift*ham2d 
  FFT_img_shift_log = 10*np.log(np.fft.fftshift(np.abs(FFT_img)))*ham2d 

  FFT_img_inv = np.fft.ifftshift(FFT_img_shift_mask)
  img_inv = np.fft.ifft2(FFT_img_inv)
  img_mod = np.abs(img_inv)
  return FFT_img_shift_log, img_mod


def noise(np_pixel_array, sigma):
  img = np_pixel_array
  gauss = img + np.random.normal(0,sigma,img.shape)
  img_gauss = np.clip(gauss, 0, 255)
  return img_gauss


In [ ]:
resultados = []
n = 1 #1, 1,100 ,10
jump = 0.1 # 0.1, 0.1, 10, 1
y_test = np.load('../input/brain-tumor-dataset-modified/y_test.npy', allow_pickle = True)

for i in tqdm(range(0,10,1)):
    if jump == 0.1:
        j = i/10
    if jump == 10:
        j = i*10
    if jump == 1:
        j = i
    x_test_data = []
    
    for number in tqdm(range(0,613)):
        im = cv2.imread('../input/images-masks/X_test_png/image0' + str(number) + '.png',0)
        im = cv2.resize(im, (256,256), interpolation = cv2.INTER_AREA)
        _, im = NEX(im,j) # mask percentage
        #_, im = cut_sup_inf(im, j) # mask percentage
        #_, im = low_pass(im, j) #size of window
        #im = noise(im, j) # sigma
        x_test_data.append(im)
        
    BATCH_SIZE = 32
    LR = 0.0001
    EPOCHS = 100
    optim = keras.optimizers.Adam(learning_rate=LR)
    dice_loss = sm.losses.DiceLoss()
    metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5), "accuracy"]
    x_test_data = np.array(x_test_data).reshape((len(x_test_data), 256, 256, 1)).astype('float16') / 255
    dependencies = {'dice_loss': dice_loss,'iou_score': sm.metrics.IOUScore(threshold=0.5), 'f1-score': sm.metrics.FScore(threshold=0.5)}
    model_= tf.keras.models.load_model('../input/segmentation-networks/MRI_mobilenet(1).h5', custom_objects=dependencies)
    model_.compile(loss=dice_loss, optimizer=optim, metrics=metrics)
    precision = model_.evaluate(x_test_data,
                                y_test_data,
                                batch_size = BATCH_SIZE)


    resultados.append(precision)
resultados = np.array(resultados)
matrix={}
matrix['loss']= resultados[:,0]
matrix['IOU']= resultados[:,1]
matrix['F1']= resultados[:,2]
matrix['Acc']= resultados[:,3]
results = pd.DataFrame(matrix)

results.to_excel("NEX_Mobilenet_seg.xlsx")  

In [ ]:
resultados = []
n = 1 #1, 1,100 ,10
jump = 0.1 # 0.1, 0.1, 10, 1
y_test = np.load('../input/brain-tumor-dataset-modified/y_test.npy', allow_pickle = True)

for i in tqdm(range(0,10,1)):
    if jump == 0.1:
        j = i/10
    if jump == 10:
        j = i*10
    if jump == 1:
        j = i
    x_test_data = []
    
    for number in tqdm(range(0,613)):
        im = cv2.imread('../input/images-masks/X_test_png/image0' + str(number) + '.png',0)
        im = cv2.resize(im, (256,256), interpolation = cv2.INTER_AREA)
        #_, im = NEX(im,j) # mask percentage
        _, im = cut_sup_inf(im, j) # mask percentage
        #_, im = low_pass(im, j) #size of window
        #im = noise(im, j) # sigma
        x_test_data.append(im)
        
    BATCH_SIZE = 32
    LR = 0.0001
    EPOCHS = 100    
    optim = keras.optimizers.Adam(learning_rate=LR)
    dice_loss = sm.losses.DiceLoss()
    metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5), "accuracy"]
    x_test_data = np.array(x_test_data).reshape((len(x_test_data), 256, 256, 1)).astype('float16') / 255
    dependencies = {'dice_loss': dice_loss,'iou_score': sm.metrics.IOUScore(threshold=0.5), 'f1-score': sm.metrics.FScore(threshold=0.5)}
    model_= tf.keras.models.load_model('../input/segmentation-networks/MRI_mobilenet(1).h5', custom_objects=dependencies)
    model_.compile(loss=dice_loss, optimizer=optim, metrics=metrics)
    precision = model_.evaluate(x_test_data,
                                y_test_data,
                                batch_size = BATCH_SIZE)


    resultados.append(precision)
resultados = np.array(resultados)
matrix={}
matrix['loss']= resultados[:,0]
matrix['IOU']= resultados[:,1]
matrix['F1']= resultados[:,2]
matrix['Acc']= resultados[:,3]
results = pd.DataFrame(matrix)

results.to_excel("CUT_SUP_INF_Mobilenet_seg.xlsx")  

In [ ]:
resultados = []
n = 100 #1, 1,100 ,10
jump = 10 # 0.1, 0.1, 10, 1
y_test = np.load('../input/brain-tumor-dataset-modified/y_test.npy', allow_pickle = True)

for i in tqdm(range(0,10,1)):
    if jump == 0.1:
        j = i/10
    if jump == 10:
        j = i*10
    if jump == 1:
        j = i
    x_test_data = []
    
    for number in tqdm(range(0,613)):
        im = cv2.imread('../input/images-masks/X_test_png/image0' + str(number) + '.png',0)
        im = cv2.resize(im, (256,256), interpolation = cv2.INTER_AREA)
        #_, im = NEX(im,j) # mask percentage
        #_, im = cut_sup_inf(im, j) # mask percentage
        _, im = low_pass(im, j) #size of window
        #im = noise(im, j) # sigma
        x_test_data.append(im)
        
    BATCH_SIZE = 32
    LR = 0.0001
    EPOCHS = 100
    optim = keras.optimizers.Adam(learning_rate=LR)
    dice_loss = sm.losses.DiceLoss()
    metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5), "accuracy"]
    x_test_data = np.array(x_test_data).reshape((len(x_test_data), 256, 256, 1)).astype('float16') / 255
    dependencies = {'dice_loss': dice_loss,'iou_score': sm.metrics.IOUScore(threshold=0.5), 'f1-score': sm.metrics.FScore(threshold=0.5)}
    model_= tf.keras.models.load_model('../input/segmentation-networks/MRI_mobilenet(1).h5', custom_objects=dependencies)
    model_.compile(loss=dice_loss, optimizer=optim, metrics=metrics)
    precision = model_.evaluate(x_test_data,
                                y_test_data,
                                batch_size = BATCH_SIZE)


    resultados.append(precision)
resultados = np.array(resultados)
matrix={}
matrix['loss']= resultados[:,0]
matrix['IOU']= resultados[:,1]
matrix['F1']= resultados[:,2]
matrix['Acc']= resultados[:,3]
results = pd.DataFrame(matrix)

results.to_excel("LOWPASS_Mobilenet_seg.xlsx")  

In [ ]:
resultados = []
n = 100 #1, 1,100 ,100
jump = 10 # 0.1, 0.1, 10, 10
y_test = np.load('../input/brain-tumor-dataset-modified/y_test.npy', allow_pickle = True)

for i in tqdm(range(0,10,1)):
    if jump == 0.1:
        j = i/10
    if jump == 10:
        j = i*10
    if jump == 1:
        j = i
    x_test_data = []
    
    for number in tqdm(range(0,613)):
        im = cv2.imread('../input/images-masks/X_test_png/image0' + str(number) + '.png',0)
        im = cv2.resize(im, (256,256), interpolation = cv2.INTER_AREA)
        #_, im = NEX(im,j) # mask percentage
        #_, im = cut_sup_inf(im, j) # mask percentage
        #_, im = low_pass(im, j) #size of window
        im = noise(im, j) # sigma
        x_test_data.append(im)

    BATCH_SIZE = 32
    LR = 0.0001
    EPOCHS = 100
    optim = keras.optimizers.Adam(learning_rate=LR)
    dice_loss = sm.losses.DiceLoss()
    metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5), "accuracy"]
    x_test_data = np.array(x_test_data).reshape((len(x_test_data), 256, 256, 1)).astype('float16') / 255
    dependencies = {'dice_loss': dice_loss,'iou_score': sm.metrics.IOUScore(threshold=0.5), 'f1-score': sm.metrics.FScore(threshold=0.5)}
    model_= tf.keras.models.load_model('../input/segmentation-networks/MRI_mobilenet(1).h5', custom_objects=dependencies)
    model_.compile(loss=dice_loss, optimizer=optim, metrics=metrics)
    precision = model_.evaluate(x_test_data,
                                y_test_data,
                                batch_size = BATCH_SIZE)


    resultados.append(precision)
resultados = np.array(resultados)
matrix={}
matrix['loss']= resultados[:,0]
matrix['IOU']= resultados[:,1]
matrix['F1']= resultados[:,2]
matrix['Acc']= resultados[:,3]
results = pd.DataFrame(matrix)

results.to_excel("NOISE_Mobilenet_seg.xlsx")  

Fazendo imagens para o artigo

In [ ]:
im = cv2.imread('../input/images-masks/X_test_png/image0227.jpg',0)
im = cv2.resize(im, (256,256), interpolation = cv2.INTER_AREA)


In [ ]:
im = cv2.imread('../input/caso-uh/IMG-0001-00006.png',0)
im = cv2.resize(im, (256,256), interpolation = cv2.INTER_AREA)
ma = cv2.imread('../input/images-masks/y_test_png/image0227.png',0)
ma = cv2.resize(ma, (256,256), interpolation = cv2.INTER_AREA)

im = np.array(im).reshape((1, 256, 256, 1)).astype('float16') #/ 255
ma = np.array(ma).reshape((1, 256, 256, 1)).astype('float16') #/ 255


LR = 0.0001
optim = keras.optimizers.Adam(learning_rate=LR)
dice_loss = sm.losses.DiceLoss()
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5), "accuracy"]
dependencies = {'dice_loss': dice_loss,'iou_score': sm.metrics.IOUScore(threshold=0.5), 'f1-score': sm.metrics.FScore(threshold=0.5)}
model_= tf.keras.models.load_model('../input/segmentation-networks/MRI_vgg16(2).h5', custom_objects=dependencies)
model_.compile(loss=dice_loss, optimizer=optim, metrics=metrics)

predicted = model_.predict(im.astype(np.float16))
predicted = np.squeeze(predicted.astype(np.float16))
plt.imsave('predicted_000.jpg', predicted, cmap='gray')
predicted_000 = cv2.imread('./predicted_000.jpg',0)

im = cv2.imread('../input/caso-uh/IMG-0001-00006.png',0)
im = cv2.resize(im, (256,256), interpolation = cv2.INTER_AREA)
stack = np.hstack((im,predicted_000))
cv2.imwrite('stack.png',stack)

fig = plt.subplots(figsize=(10,26))
stack = cv2.imread('./stack.png',1)
plt.imshow(stack)
plt.show()

In [ ]:
fig = plt.subplots(figsize=(10,10))
plt.imshow(cv2.addWeighted( im, 0.6, predicted_000, 0.9,0.0),cmap='gray')
plt.show()
cv2.imwrite('sum.png',cv2.addWeighted( im, 0.6, predicted_000, 0.9,0.0))

In [ ]:
fig = plt.subplots(figsize=(10,10))
plt.imshow(predicted_000,cmap='gray')
plt.show()
cv2.imwrite('predicted_000.png',predicted_000)

# 0. Interesting things

In [ ]:
# Read .mat and convert to numpy
#Extra: data_dict.key() --> See the key

data_dict = mat73.loadmat('../input/brain-tumor-dataset-original/brainTumorDataPublic_1-766/1.mat')
imgplot = plt.imshow(data_dict['cjdata']['image'], cmap = 'gray')

In [ ]:
# Read .mat and convert to numpy
#Extra: data_dict.key() --> See the key

data_dict = mat73.loadmat('../input/brain-tumor-dataset-original/brainTumorDataPublic_1-766/1.mat')
imgplot = plt.imshow(data_dict['cjdata']['tumorMask'], cmap = 'gray')

In [ ]:
# Read .mat and convert to numpy
#Extra: data_dict.key() --> See the key

data_dict = mat73.loadmat('../input/brain-tumor-dataset-original/brainTumorDataPublic_1-766/1.mat')
data_dict['cjdata']['label']

In [ ]:
#https://www.kaggle.com/vbookshelf/play-audio-read-the-files-create-a-spectrogram

from pydub import AudioSegment
import IPython

path = '../input/alarm-sound/Alarm_Slow_A1_fesliyanstudios.mp3'

IPython.display.Audio(path, autoplay=True)